In [ ]:
# Searchlight parameters
sl_rad = 2
max_blk_edge = 10
pool_size = 1

In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image
import logging
from brainiak.searchlight.searchlight import Searchlight

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
import sys
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
localizer = 'Scramble'  # scramble or object. This is the localizer task.
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
sub_info = sub_info[sub_info['group'] == 'control']
subs = sub_info['sub'].tolist()

run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

# New parameter for searchlight
searchlight_radius = 2  # in voxels, adjust as needed

def load_and_prepare_data(sub, run_combo):
    """
    Load and prepare data for a single subject and run combination.
    
    :param sub: Subject ID
    :param run_combo: List of run numbers to combine
    :return: Tuple of (4D fMRI data, brain mask, psychological covariate)
    """
    logging.info(f"Loading data for subject {sub}, runs {run_combo}")
    
    # Load and combine run data
    run_data_list = []
    for run in run_combo:
        run_file = f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-0{run}/1stLevel.feat/filtered_func_data_reg.nii.gz'
        run_img = image.load_img(run_file)
        run_data = image.clean_img(run_img, standardize=True)
        run_data_list.append(run_data)
    
    # Concatenate run data
    fmri_data = image.concat_imgs(run_data_list)
    
    # Load brain mask
    mask_file = f'{study_dir}/{sub}/ses-01/derivatives/rois/pIPS.nii.gz'
    brain_mask = nib.load(mask_file).get_fdata().astype(bool)
    
    # Generate psychological covariate
    psy = make_psy_cov(run_combo, sub)
    
    # Ensure fMRI data and psy covariate have the same number of time points
    if fmri_data.shape[-1] != len(psy):
        raise ValueError(f"Mismatch in volumes: fMRI data has {fmri_data.shape[-1]}, psy has {len(psy)}")
    
    return fmri_data.get_fdata(), brain_mask, psy

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols_per_run, tr = 184, 2.0
    total_vols = vols_per_run * len(runs)
    times = np.arange(0, total_vols * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for i, rn in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_{localizer}.txt'

        if not os.path.exists(obj_cov_file):
            logging.warning(f'Covariate file not found for run {rn}')
            continue

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        
        if i > 0:
            obj_cov['onset'] += i * vols_per_run * tr
        
        full_cov = pd.concat([full_cov, obj_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        logging.warning('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((total_vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    psy[psy > 0] = 1
    psy[psy <= 0] = 0
    return psy

# Main execution
if __name__ == "__main__":
    for sub in subs:
        for run_combo in run_combos:
            try:
                fmri_data, brain_mask, psy = load_and_prepare_data(sub, run_combo)
                logging.info(f"Data loaded successfully for subject {sub}, runs {run_combo}")
                logging.info(f"fMRI data shape: {fmri_data.shape}")
                logging.info(f"Brain mask shape: {brain_mask.shape}")
                logging.info(f"Psychological covariate shape: {psy.shape}")
                
                # Here, we'll add the searchlight analysis in the next step
                
            except Exception as e:
                logging.error(f"Error loading data for subject {sub}, runs {run_combo}: {str(e)}")

/home/csimmon2/anaconda3/envs/brainiak_env/lib/python3.7/site-packages/nilearn/__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()
2024-10-04 16:04:20,524 - INFO - Loading data for subject sub-025, runs [1, 2]
